# **Week 3 Hands-on Lab: Visualizing the Self-Attention Mechanism**

In this hands-on lab, we will visualize the self-attention mechanism in Large Language Models. This will help you understand how it works.

This hands-on lab will allow you to:
1.	Implement and visualize the self-attention mechanism using Python.
2.	Understand how attention scores are calculated and applied to input data.
3.	Gain hands-on experience in a simplified environment to appreciate the role of attention in NLP models.


# **Part 1: Compute Self-Attention Scores**

**1.	Define Input Data**

Create a toy dataset representing embeddings for a sentence.

In [ ]:
import numpy as np

# Example sentence: "The cat sat on the mat"
sentence = ["The", "cat", "sat", "on", "the", "mat"]

# Dummy embeddings for each word (6 words, 4-dimensional embeddings)
embeddings = np.random.rand(6, 4)
print("Word Embeddings:\n", embeddings)

**2.	Create Query, Key, and Value Matrices**

Generate query, key, and value vectors for each word.


In [ ]:
def generate_qkv(embeddings, d_k=4):
    np.random.seed(0)  # For reproducibility
    W_q = np.random.rand(d_k, d_k)  # Query weights
    W_k = np.random.rand(d_k, d_k)  # Key weights
    W_v = np.random.rand(d_k, d_k)  # Value weights
    Q = np.dot(embeddings, W_q)
    K = np.dot(embeddings, W_k)
    V = np.dot(embeddings, W_v)
    return Q, K, V

Q, K, V = generate_qkv(embeddings)
print("Query Vectors:\n", Q)
print("Key Vectors:\n", K)
print("Value Vectors:\n", V)

**3.	Compute Attention Scores**

Calculate attention scores using the dot product of queries and keys, followed by a softmax function.


In [ ]:
def attention_scores(Q, K):
    d_k = Q.shape[1]  # Dimension of key vectors
    scores = np.dot(Q, K.T) / np.sqrt(d_k)  # Scaled dot product
    softmax_scores = np.exp(scores) / np.sum(np.exp(scores), axis=1, keepdims=True)
    return softmax_scores

attention = attention_scores(Q, K)
print("Attention Scores:\n", attention)

**4.	Apply Attention Scores to Value Vectors**

Multiply attention scores with the value vectors to compute the final outputs.


In [ ]:
def attention_output(attention, V):
    return np.dot(attention, V)

output = attention_output(attention, V)
print("Self-Attention Output:\n", output)

# **Part 2: Visualize Attention**

1.	**Create a Heatmap of Attention Scores**

Use matplotlib to visualize the attention scores between words.


In [ ]:
import matplotlib.pyplot as plt

def plot_attention(sentence, attention):
    plt.figure(figsize=(8, 6))
    plt.imshow(attention, cmap='viridis')
    plt.colorbar()
    plt.xticks(ticks=range(len(sentence)), labels=sentence, rotation=90)
    plt.yticks(ticks=range(len(sentence)), labels=sentence)
    plt.title("Self-Attention Scores")
    plt.show()

plot_attention(sentence, attention)

# **Part 3: Experimentation**

1.	**Change Sentence Length and Dimensions**

Modify the sentence and embeddings variables to explore how attention adapts to different inputs.

2.	**Analyze Different Weight Matrices**

Experiment with different random weights for query, key, and value matrices to observe changes in attention scores.

3.	**Add Masking**

Implement masking to ignore certain positions (e.g., padding tokens) in the sequence.


In [ ]:
def masked_attention_scores(Q, K, mask=None):
    d_k = Q.shape[1]
    scores = np.dot(Q, K.T) / np.sqrt(d_k)
    if mask is not None:
        scores += (mask * -1e9)  # Add large negative value to masked positions
    softmax_scores = np.exp(scores) / np.sum(np.exp(scores), axis=1, keepdims=True)
    return softmax_scores

# Example mask: ignore the last word
mask = np.array([[0, 0, 0, 0, 0, -1]])
masked_attention = masked_attention_scores(Q, K, mask)
plot_attention(sentence, masked_attention)

# **Bonus Part 4: Try the Bertviz library to visualize attention scores in a real model**

In this part, we will experiment with the Bertviz library.
BertViz is an interactive tool for visualizing attention in Transformer language models such as BERT, GPT2, or T5. It can be run inside a Jupyter or Colab notebook through a simple Python API that supports most Huggingface models.

In [ ]:
#install the library
!pip install bertviz

First load a Huggingface model, either a pre-trained model as shown below, or your own fine-tuned model. Be sure to set output_attentions=True.

In [ ]:
from transformers import AutoTokenizer, AutoModel, utils
utils.logging.set_verbosity_error()  # Suppress standard warnings
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)

Then prepare inputs and compute attention:

In [ ]:
inputs = tokenizer.encode("The cat sat on the mat", return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(inputs[0])

Finally, display the attention weights using the [head_view](https://github.com/jessevig/bertviz/blob/master/bertviz/head_view.py) or [model_view](https://github.com/jessevig/bertviz/blob/master/bertviz/model_view.py) functions:

In [ ]:
from bertviz import head_view
#The head view visualizes attention for one or more attention heads in the same layer
head_view(attention, tokens)

In [ ]:
from bertviz import model_view
#The model view shows a bird's-eye view of attention across all layers and heads.
model_view(attention, tokens)